# ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [87]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [88]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [89]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [90]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821



The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [91]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

session = cluster.connect()

#### Create Keyspace

In [92]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS cassandra_keyspace 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [93]:
try:
    session.set_keyspace('cassandra_keyspace')
except Exception as e:
    print(e)

## The following queries answer these three questions of the data

### 1. The artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [ ]:
## Query 1: the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS session_songs "
query = query + "(sessionId bigint, itemInSession bigint, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession) )"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [95]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_songs (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [96]:
query = "SELECT artist, song, length FROM session_songs WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


The reason why I'm choosing sessionId as my partition key is because the query asks me for a specific sessionId. The clustering column (itemInSession) is also chosen because of the where statement. These two columns make a unique combination already, so I don't need any extra clustering column.

In [97]:
## Query 2: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS user_songs "
query = query + "(userId bigint, sessionId bigint, itemInSession bigint, artist text, song text, full_name text, PRIMARY KEY (userId, sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [98]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_songs (userId, sessionId, itemInSession, artist, song, full_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1] + ' ' + line[4]))

In [99]:
query = "SELECT artist, song, full_name FROM user_songs WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.full_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


The reason why I'm choosing userId as my partition key is because the query asks me for a specific userId. The clustering columns (sessionId and itemInSession) are also chosen because the WHERE clause and because If I only choose sessionId and do not take itemInSession, that would not make a unique combination.

In [100]:
## Query 3: every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "CREATE TABLE IF NOT EXISTS users_by_song "
query = query + " (song text, userId bigint, full_name text, PRIMARY KEY (song, userId)) "
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [101]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_by_song (song, userId, full_name)"
        query = query + " VALUES (%s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1] + ' ' + line[4]))

In [102]:
query = "SELECT full_name FROM users_by_song WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.full_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


The reason why I'm choosing song as my partition key is because the query is directly asking for songs in the WHERE clause. I'm also using userId as clustering column in order to make the records unique. We are only interested in checking which users have listened to a particular song, not the quantity.

### Drop the tables before closing out the sessions

In [103]:
query = "DROP TABLE session_songs"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [104]:
query = "DROP TABLE user_songs"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [105]:
query = "DROP TABLE users_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [106]:
session.shutdown()
cluster.shutdown()